This notebook is a utility to move and organize files for the block based synchronization paradigm - it is a pre-requisite for running the pipeline

In [40]:
import pathlib
import os
import shutil
import datetime
import subprocess as sp
import re
import pandas as pd
import numpy as np
import tqdm
import glob
from lxml import etree

In [114]:
# Rename AUX labeled files as CHxs
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        for i in oe_folder.iterdir():
            if 'AUX' in str(i):
                ch_num = str(i)[-12]
                ch_num = str(int(ch_num) + 32)
                destination = i.parent / (str(i.name[:4]) + 'CH' + ch_num + '.continuous')
                os.rename(i,destination)

no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK


In [113]:
# extract hidehere files back into the block:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        if (oe_folder / 'hidehere').is_dir():
            print(f'take care of block {oe_folder}')
        else:
            print(f'nothing here')


nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing

In [48]:
def produce_analysis_report(path_to_animal_folder,output_folder, animal_number):
    p = pathlib.Path(path_to_animal_folder)
    os.listdir(p)
    folders = os.listdir(p)
    ind_first_level = []
    ind_second_level = []
    condition = []
    for folder in sorted(folders):
        if 'block' not in folder:
            date_path = p / folder
            blocks = os.listdir(date_path)
            for block in blocks:
                b_dict = {
                    'automated_step': False,
                    'manual_sync':    False,
                    'bokeh_plot':     False
                }
                ind_first_level.append(date_path.name)
                ind_second_level.append(block)
                a_files = os.listdir(p / folder / block / 'analysis')
                if 'blocksync_df.csv' in a_files:
                    b_dict['automated_step'] = True
                if len([f for f in a_files if '.html' in f]) != 0:
                    b_dict['bokeh_plot'] = True
                if 'manual_sync_df.csv' in a_files:
                    b_dict['manual_sync'] = True
                condition.append(b_dict)

    arrays = [
        np.array(ind_first_level),
        np.array(ind_second_level)
    ]
    df = pd.DataFrame(data=None, index=arrays, columns=['automated_step','manual_synchronization','bokeh_plot'])
    auto_step_col = [i['automated_step'] for i in condition]
    manual_sync_col = [i['manual_sync'] for i in condition]
    bokeh_plot_col = [i['bokeh_plot'] for i in condition]
    df['automated_step'] = auto_step_col
    df['manual_synchronization'] = manual_sync_col
    df['bokeh_plot'] = bokeh_plot_col
    p = pathlib.Path(output_folder) / f'{animal_number}_analysis_report.csv'
    df.to_csv(p)

In [50]:
output_folder = pathlib.Path(r'\\sil2\Data\Nimrod\experiments')
animal_folder = pathlib.Path(r'\\sil2\data\Nimrod\experiments\PV_24')
produce_analysis_report(animal_folder,output_folder,animal_number='PV24')

In [46]:
p = pathlib.Path(r'\\sil2\data\Nimrod\experiments\PV_24')
folders = os.listdir(p)
ind_first_level = []
ind_second_level = []
condition = []
for folder in sorted(folders):
    if 'block' not in folder:
        date_path = p / folder
        blocks = os.listdir(date_path)
        for block in blocks:
            b_dict = {
                'automated_step': False,
                'manual_sync':    False,
                'bokeh_plot':     False
            }
            ind_first_level.append(date_path.name)
            ind_second_level.append(block)
            a_files = os.listdir(p / folder / block / 'analysis')
            if 'blocksync_df.csv' in a_files:
                b_dict['automated_step'] = True
            if len([f for f in a_files if '.html' in f]) != 0:
                b_dict['bokeh_plot'] = True
            if 'manual_sync_df.csv' in a_files:
                b_dict['manual_sync'] = True
            condition.append(b_dict)

arrays = [
    np.array(ind_first_level),
    np.array(ind_second_level)
]
df = pd.DataFrame(data=None, index=arrays, columns=['automated_step','manual_synchronization','bokeh_plot'])
auto_step_col = [i['automated_step'] for i in condition]
manual_sync_col = [i['manual_sync'] for i in condition]
bokeh_plot_col = [i['bokeh_plot'] for i in condition]
df['automated_step'] = auto_step_col
df['manual_synchronization'] = manual_sync_col
df['bokeh_plot'] = bokeh_plot_col
df

automated_step  manual_synchronization  bokeh_plot
02_01_2022 block_37            True                   False       False
           block_38            True                   False       False
           block_39           False                   False       False
           block_40           False                   False       False
           block_41            True                   False       False
...                             ...                     ...         ...
29_12_2021 block_32           False                   False       False
           block_33           False                   False       False
30_12_2021 block_34           False                   False       False
           block_35            True                   False       False
           block_36           False                   False       False

[92 rows x 3 columns]

In [20]:
arrays= [
    np.array(['02_03_2022','02_03_2022','02_03_2022']),
    np.array(['block_1','block_2','block_2'])
]
df = pd.DataFrame(data=None,index=arrays,columns=['Block'])
df

Block
02_03_2022 block_1   NaN
           block_2   NaN
           block_2   NaN

In [79]:
# reptilearn data sorter draft
p = pathlib.Path(r"D:\experiments\Transients\reptilearn_recordings")
files = os.listdir(p)
for file in files:
        name = file[-19:]
        pre_name = file[:-19]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:
            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_right_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Trans

In [17]:
def reptilearn_data_sorter(path_to_dir):
    """
    This function takes a bunch of reptilearn data and sorts them by date and time as per the given format: SomeTitle_CameraName_YYYYMMDD_HHMMSS.mp4 or csv and sorts them as follows:
     date_folder --> Time_folder --> files
    :param path_to_dir: Pathlib.Path object which points to the directory of choice
    :return:
    """
    p = path_to_dir
    files = os.listdir(p)
    for file in files:
        if os.path.isdir(p / file):
            continue
        name = file[-19:]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        #print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:

            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

In [18]:
reptilearn_data_sorter(pathlib.Path(r'D:\experiments\Transients\reptilearn_recordings'))

destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination 

In [166]:
def reptilearn_organizer_date_to_date(date, path_to_animal_folder, path_to_arena_files):
    """
    This function takes two date folders (with the exact same date format and values) and matches the arena files to OE time signatures - This should be done after all files are organized into date folders on both sides and Open ephys files are in their correct place within the block tree
    :param date: DD_MM_YYYY
    :param path_to_animal_folder: the path to the animal folder (for example, "D:\experiments\PV_24")
    :param path_to_arena_files:  the path to the reptilearn files folder (for example "D:\experiments\Transients\reptilearn_recordings"
    :return: None
    """
    p_dst = pathlib.Path(rf'{path_to_animal_folder}\{date}') #path to a specific date in the animal folder
    p_ori = pathlib.Path(rf'{path_to_arena_files}\{date}') # path to the arena files sorted by date
    dst_ls = os.listdir(p_dst)
    ori_ls = os.listdir(p_ori)
    dst_times = []
    for file in dst_ls:
        p_oe = p_dst / file / 'oe_files'
        filename = os.listdir(p_oe)
        if len(filename) != 1:
            print(f'Missing open ephys file in {file}, cannot work automatically')
            continue
        oe_time = filename[0][-8:]
        t = datetime.timedelta(hours=int(oe_time[0:2]),
                               minutes=int(oe_time[3:5]),
                               seconds=int(oe_time[6:8]))
        dst_times.append([file,t])

    if len(ori_ls) == len(dst_ls):
        print(f'same file count')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
    #            print(f'{dst_times[i][0]} => {delta.seconds}')
            block_position = np.argmin(deltas)
            dst_block = dst_times[block_position][0]
            files_to_move = [p_ori / file / f for f in os.listdir(p_ori / file)]
            print(f'will move {arena_time} to {p_dst / dst_block}')
            print(f'because the delta is {deltas[block_position]}')
            for f in files_to_move:
                shutil.move(f,p_dst / dst_block / "arena_videos" / f.name)
                print(f'moved {f} to {p_dst / dst_block /"arena_videos"/f.name}')
    else:
        print('The file count does not match - work manually you lazy bum')
        print('okay, here are the time differences:')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            print(f'these are the deltas for {file}')
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
                print(f'{dst_times[i][0]} => {delta.seconds}')

In [167]:
reptilearn_organizer_date_to_date('04_01_2022', r"D:\experiments\PV_24", r"D:\experiments\Transients\reptilearn_recordings")

Missing open ephys file in block_65, cannot work automatically
same file count
will move 10:41:39 to D:\experiments\PV_24\04_01_2022\block_60
because the delta is 8
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\right_20220104-104139.csv to D:\exper

In [122]:
def build_file_tree(path, blocks_to_build, animal_name_number):
    """
    This function creates the empty file trees required to populate later
    :param path: pathlib.Path location to start the tree
    :param blocks_to_build: an iterable containing block numbers to build
    :param animal_name_number: the name of the folder where the block trees will be created
    """
    #build tree root for specific animal
    try:
        tree_root = pathlib.Path(path / animal_name_number)
        os.mkdir(tree_root)
    except FileExistsError:
        print('Animal folder already exists')
    #create a block template folder
    try:
        block_root = pathlib.Path(tree_root / 'block_template')
        os.mkdir(block_root)
        #make the block template itself:
        os.mkdir(block_root / 'analysis')
        os.mkdir(block_root / 'oe_files')
        os.mkdir(block_root / 'eye_videos')
        os.mkdir(block_root / 'eye_videos' / 'LE')
        os.mkdir(block_root / 'eye_videos' / 'RE')
        os.mkdir(block_root / 'arena_videos')
    except FileExistsError:
        print('template tree already exists')
    #duplicate the template to the number of blocks needed:
    for i in blocks_to_build:
        try:
            folder_to_create = tree_root / f'block_{i}'
            shutil.copytree(block_root,folder_to_create)
            print(f'created {folder_to_create}')
        except FileExistsError:
            print(f'block number {i} already there')

In [179]:
build_file_tree(pathlib.Path(r"D://experiments"),
                range(32,94),
                "PV_24")

Animal folder already exists
template tree already exists
block number 32 already there
block number 33 already there
block number 34 already there
block number 35 already there
block number 36 already there
block number 37 already there
block number 38 already there
block number 39 already there
block number 40 already there
block number 41 already there
block number 42 already there
block number 43 already there
block number 44 already there
block number 45 already there
block number 46 already there
block number 47 already there
block number 48 already there
block number 49 already there
block number 50 already there
block number 51 already there
block number 52 already there
block number 53 already there
block number 54 already there
block number 55 already there
block number 56 already there
block number 57 already there
block number 58 already there
block number 59 already there
block number 60 already there
block number 61 already there
block number 62 already there
block number

In [188]:
# open ephys file mover VERSION1
#initialize
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
path_to_oe_data = pathlib.Path(r'\\sil1\Data\Pogona Vitticeps\PV24')
folders_to_move = [i for i in os.listdir(path_to_oe_data) if 'EE' in i]

#move open ephys files
for folder in folders_to_move:
    s = folder[folder.find('EE'):]
    boolean_digit_locations = [b.isdigit() for b in s]
    boolean_underscore_locations = [b == '_' for b in s]
    fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
    funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
    if fd_inds[0] < funder_inds[0]:
        block_num = folder[fd_inds[0]:funder_inds[0]]
    else:
        block_num = folder[fd_inds[0]:funder_inds[1]]
    if int(block_num) > 32:
        #print(f'folder {folder} will go to block number {block_num}')
        origin = pathlib.Path(path_to_oe_data / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files' / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE68_2022-01-05_14-40-31 to D:\experiments\PV_24\block_68\oe_files\PV24_05_01_22_EE68_2022-01-05_14-40-31
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE69_2022-01-05_15-12-37 to D:\experiments\PV_24\block_69\oe_files\PV24_05_01_22_EE69_2022-01-05_15-12-37
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE70_2022-01-05_15-48-24 to D:\experiments\PV_24\block_70\oe_files\PV24_05_01_22_EE70_2022-01-05_15-48-24
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE71_2022-01-05_16-20-18 to D:\experiments\PV_24\block_71\oe_files\PV24_05_01_22_EE71_2022-01-05_16-20-18
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE72_2022-01-05_16-44-09 to D:\experiments\PV_24\block_72\oe_files\PV24_05_01_22_EE72_2022-01-05_16-44-09
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_10_01_22_EE73_2022-01-10_13-55-31 to D:\experiments\PV_24\block_73\oe_files\PV24_10_01_22_EE73_2022-01-10_13-55-31
Copying \\sil1\Data\Pogona V

In [ ]:
# open ephys file mover VERSION2

In [126]:
# Move eye camera files __VERSION1__
# Initialize:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for s in folders_to_move:
        l = s.find('EE')
        #print(s[l+2:l+5])
        if s[l:l+2] == 'EE_':
            block_num =
        elif s[l+2] == '_':
            #print(f'the block number for {s} is {s[l+3]}')
            block_num = s[l+3]
        elif s[l+3] == '_':
            #print(f'the block number for {s} is {s[l+2]}')
            block_num = s[l+2]
        elif s[l+4] == '_':
            #print(f'the block number for {s} is {s[l+2:l+4]}')
            block_num = s[l+2:l+4]
        origin = pathlib.Path(eye / s)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / s)
        print(f'Copying {origin} to {destination}')
        try:
            #shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE37_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_37\eye_videos\LE\PV24_EE37_640x480_60hz_experiment_1_re

In [180]:
#eye video block identifier + mober Version 2

path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for folder in folders_to_move:
        s = folder[folder.find('EE'):]
        boolean_digit_locations = [b.isdigit() for b in s]
        boolean_underscore_locations = [b == '_' for b in s]
        fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
        funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
        if fd_inds[0] < funder_inds[0]:
            block_num = folder[fd_inds[0]:funder_inds[0]]
        else:
            block_num = folder[fd_inds[0]:funder_inds[1]]
        origin = pathlib.Path(eye / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Trans

In [84]:
#Dating the blocks:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
block_num = 1
oe_path = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files')
s = os.listdir(oe_path)[0]
date_starts = s.find("2021")
date = s[date_starts:date_starts +10]
date


'2021-12-23'

In [71]:
def data_mover_UNFINISHED(path_to_tree_root,path_to_data_source,identifier,data_type):
    """
    This function takes data from a source into the tree by identifier and type
    :param path_to_tree_root:
    :param path_to_data_source:
    :param identifier:
    :param data_type:
    :return:
    """
    #list the directories in the data source
    folders_to_move = [i for i in os.listdir(path_to_data_source) if identifier in i]

    #find the block number
    for s in folders_to_move:
        l = s.find(identifier)
        #print(s[l+2:l+5])
        if s[l+2] == '_':
            #print(f'the block number for {s} is {s[l+3]}')
            block_num = s[l+3]
        elif s[l+3] == '_':
           #print(f'the block number for {s} is {s[l+2]}')
            block_num = s[l+2]
        elif s[l+4] == '_':
            #print(f'the block number for {s} is {s[l+2:l+4]}')
            block_num = s[l+2:l+4]

a
v
s


In [108]:
#cleanup_DLC_files
def cleanup_dlc_files(block_path):
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
        path = block_path / 'eye_videos' / prefixer
        vid_folder = os.listdir(path)
        path = path / vid_folder[0]
        files_to_save = [path / i for i in os.listdir(path) if 'labeled.mp4' in i]
        for file in files_to_save:
            shutil.move(pathlib.Path(file),path / 'analysis' )
        files_to_remove = [path / i for i in os.listdir(path) if 'DLC' in i]
        for file in files_to_remove:
            os.remove(file)


In [109]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    cleanup_dlc_files(block_path)

In [120]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
       path = block_path / 'eye_videos' / prefixer
       vid_folder = os.listdir(path)
       path = path / vid_folder[0]
       file_name = [i for i in os.listdir(path) if 'oldLabeled.mp4' in i][0]
       file_to_move = path / file_name
       shutil.move(file_to_move, block_path / 'analysis' / file_name)
       #os.rename(path / 'analysis',path / f'{base_name[:-4]}_oldLabeled.mp4')
